# Page Rank Implementation

PageRank is the underlying algorithm used by Google to rank websites in their search engine results. PageRank works by counting the number of links to a page to determine a ranking for the website. Higher-ranked websites have more links from other websites.

This is a implementation of the Page Rank algorithm using Markov Chains.  Of all the implementations out there, I found the implementation by Daniel Bartolomé to be clean and computationally efficient.  It has been modified for better control on convergence and to return rank and convergence information.

To use PageRank to rank websites, model the internet as a graph where websites are represented as vertices and edges are hyperlinks. The algorithm inputs the adjacency matrix corresponding to this graph and computes the PageRank score per vertex using Markov Chains. It outputs a descending ranked list of vertices (ranked websites).

**Reference**
*Daniel Bartolomé. Github. PageRank Implementation in R and Python. Last Accessed July 2018. [https://github.com/BartolomeD/pagerank](https://github.com/BartolomeD/pagerank).*

*Sergei Brin and Lawrence Page. (1998). "The anatomy of a large-scale hypertextual Web search engine" (PDF). Computer Networks and ISDN Systems. 30: 107–117. Accessed July 2018. [http://infolab.stanford.edu/pub/papers/google.pdf](http://infolab.stanford.edu/pub/papers/google.pdf).*

### Update the environment

In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /opt/conda/lib/python3.5/site-packages (18.0)


In [2]:
!pip install -U sklearn

Requirement already up-to-date: sklearn in /opt/conda/lib/python3.5/site-packages (0.0)


In [3]:
!pip install -U numpy

Requirement already up-to-date: numpy in /opt/conda/lib/python3.5/site-packages (1.15.0)


In [4]:
!pip install -U pandas

Requirement already up-to-date: pandas in /opt/conda/lib/python3.5/site-packages (0.23.3)


### Determine the Underlying environment

In [5]:
env = None
colaboratory_flag = False
try:
    # This will work for everything but Google Colaboratory since the utility is somewhere on Google Drive
    # which is not yet mounted and so it is not reachable.
    #
    from utilities.detect_environment import detect_environment

    env = detect_environment()

    print('Detected Environment: {}'.format(env))
    if env.detected_environment == utilities.detect_environment.COLABORATORY:
        colaboratory_flag = True
    
except:
    # Check if on Colaboratory
    import os
    try:
        os.environ['DATALAB_SETTINGS_OVERRIDES']
        colaboratory_flag = True
    except:
        pass

Detected Environment: Local


Mount Google Drive if on Colaboratory.

In [6]:
if colaboratory_flag:
    #colab
    # google-drive-ocamlfuse
    # https://github.com/astrada/google-drive-ocamlfuse
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
    !apt-get update -qq 2>&1 > /dev/null
    !apt-get -y install -qq google-drive-ocamlfuse fuse

    # Colab Auth token
    from google.colab import auth
    auth.authenticate_user()

    # Drive FUSE library credential
    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

    # drive/ Google Drive
    !mkdir -p drive
    !google-drive-ocamlfuse drive

### Set up logging

In [7]:
import logging
import utilities.setup_logging

utilities.setup_logging.setup_logging()

## PageRank Algorithm using Markov Chains

The theory behind the PageRank algorithm holds that an imaginary surfer who is randomly surfing pages through links will eventually stop clicking. The probability at any step that the person will continue is representing as a damping factor. Various studies have tested different damping factors, but it is generally assumed that the damping factor will be set around 0.85.  Consider the implementation is the Markov chain:

```python
        for j in range(len(t)):
            if sum(a[j, :]) == 0:
                t[j] = 1 / ncol
            else:
                t[j] = (1-damping) / ncol + damping * sum(a[:, j] * (s[i, :] / np.sum(a, 1)))
```

`j` cycles over all the rows in the matrix.  For the jth row, if the sum of the row is zero, then initialize the influence to be equal ($\frac{1}{N}$) and sum to one (probabilistic).

For the jth row which has a sum not equal to zero, then the update rule at step ```j``` is given as a recursive probability $PR$ (the Markov Chain):

$$PR(p_i;t+1) = \frac{1-d}{N} + d \sum_{p_i \in M(p_i)} \frac{PR(p_j, t)}{L(p_j)}$$

where $N$ is the number of items, $d$ is the damping factor, and $L$ is the number of outbound links.  It is expressed in the else clause above.

**References**
*PageRank. Wikipedia. Accessed July 2018. [https://en.wikipedia.org/wiki/PageRank](https://en.wikipedia.org/wiki/PageRank).

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


def pagerank(a, damping=0.85, convergence_tolerance=1.0e-9):
    
    # initialize transition matrix
    ncol = a.shape[1]
    logging.info('Operating on {} entities'.format(ncol))
    # s is a row vector with even weighting between the ncol entities
    s = np.repeat(1 / ncol, ncol).reshape(1, -1)
    # initialize the counter
    i = 0
    
    # run markov chain
    while True:
        
        # transition vector at t + 1
        t = np.empty(ncol)  # Start with an empty vector
        for j in range(len(t)):
            if sum(a[j, :]) == 0:
                t[j] = 1 / ncol
            else:
                t[j] = (1-damping) / ncol + damping * sum(a[:, j] * (s[i, :] / np.sum(a, 1)))
            

        s = np.vstack([s, t])

        i += 1
        err = mean_squared_error(s[i - 1, :], s[i, :])
        logging.info('Iteration {}: Approximate Error {}'.format(i, err))
        
        # break if converged
        if (i > 0) and (err <= convergence_tolerance):
            break
    
    # sort nodes
    out = pd.Series(np.round(s[-1, :], 4)).reset_index().sort_values(0)[::-1].values
    
    # Gather sorted pagerank scores
    scores = []
    for node in range(ncol):
        scores.append((int(out[node, 0]), np.round(out[node, 1], 4)))
    return scores, i+1, err

/opt/conda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [9]:
numpages = 10
a = np.random.randint(0, 2, (numpages, numpages))
print('Example Input')
print('-------------')
print(a)

Example Input
-------------
[[0 1 0 1 1 0 0 1 1 0]
 [0 0 1 1 1 0 1 1 1 0]
 [1 1 1 1 0 0 1 0 1 1]
 [0 1 1 1 0 1 0 0 0 1]
 [0 1 1 0 1 1 0 0 1 0]
 [0 0 0 0 1 0 0 1 1 1]
 [0 0 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 0 0 1]
 [1 0 1 1 0 0 1 0 1 0]
 [1 0 0 1 0 1 0 1 0 1]]


In [10]:
scores, numiterations, error = pagerank(a)

INFO:root:Operating on 10 entities
INFO:root:Iteration 1: Approximate Error 0.0003780591836734694
INFO:root:Iteration 2: Approximate Error 5.001978882351782e-05
INFO:root:Iteration 3: Approximate Error 1.9634518917326716e-06
INFO:root:Iteration 4: Approximate Error 1.3804103089795372e-07
INFO:root:Iteration 5: Approximate Error 1.143644825695597e-08
INFO:root:Iteration 6: Approximate Error 4.827463122484251e-10


In [11]:
print('')
print('Converted on iteration {} with error {}'.format(numiterations, error))
print('Item     Score')
for item in scores:
    print('{:4}  {:8}'.format(item[0], item[1]))



Converted on iteration 7 with error 4.827463122484251e-10
Item     Score
   3    0.1311
   2     0.124
   8    0.1205
   9    0.1094
   4    0.0973
   1    0.0951
   7     0.087
   0    0.0815
   5    0.0813
   6    0.0729
